# Transportation and Distribution

**Table of contents**

- Overview
- Setup
  - Authentication Token
- Query
  - Output Description
- Related Links

## Overview

Scope 3 Emissions are greenhouse gas emissions from upstream or downstream in the value chain of a company, for example, business travel or the transportation of goods.

Use the Transport API to calculate emissions from business-related employee travel, employee commuting, and freight transport. 

## Setup

Ensure that Python 3+ is installed on your system.


<b>Note:</b> To run this notebook, you must first add your credentials to `'../../../auth/secrets.ini'` in the following format:

```
[EAPI]
api.api_key = <Your Emissions API Key>
api.tenant_id = <Your Emissions API Tenant Id>
api.org_id = <Your Emissions API Org Id>

```

In [ ]:
# Install the prerequisite Python packages.
%pip install pandas configparser IPython requests

In [1]:
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

### Authentication Token

Run the following code snippet to generate a Bearer Token by using your api_key configured in secrets.ini.

In [2]:
config = configparser.RawConfigParser()
config.read(['../../../auth/secrets.ini','../../../auth/config.ini'])

EAPI_API_KEY        = config.get('EAPI', 'api.api_key')
EAPI_TENANT_ID      = config.get('EAPI', 'api.tenant_id')
EAPI_CLIENT_ID      = 'ghgemissions-' + EAPI_TENANT_ID
EAPI_ORG_ID         = config.get('EAPI', 'api.org_id')

EAPI_AUTH_CLIENT_ID = 'saascore-' + EAPI_TENANT_ID
EAPI_AUTH_ENDPOINT  = config.get('EAPI', 'api.auth_endpoint')

EAPI_BASE_URL       = config.get('EAPI', 'api.base_url')
EAPI_ENDPOINT       = f"{EAPI_BASE_URL}/transportation-and-distribution"

auth_request_headers: dict = {}
auth_request_headers["X-IBM-CLIENT-ID"] = EAPI_AUTH_CLIENT_ID
auth_request_headers["X-API-KEY"] = EAPI_API_KEY

verify = True

auth_url = f"{EAPI_AUTH_ENDPOINT}?orgId={EAPI_ORG_ID}"
              
response = requests.get(url = auth_url,
                        headers = auth_request_headers,
                        verify  = verify
                       )
if response.status_code == 200:
    jwt_token = response.text
    print("Authentication Success")
else:     
    print("Authentication Failed")
    print(response.text)

Authentication Success


## Query

The example request payload (below) queries IBM Envizi - Emissions API for a short haul business flight taken from the United States of 1000 miles:

In [3]:
payload = {
  "activity": {
    "type": "Business flights:Short Haul",
    "value": 1000,
    "unit": "mi"
  },
  "location": {
    "country": "USA"
  },
  "time": {
    "date": "2025-01-01"
  }
}

In [4]:
# Create the query headers
request_headers: dict = {}
request_headers["Content-Type"] = "application/json"
request_headers["x-ibm-client-id"] = EAPI_CLIENT_ID
request_headers["Authorization"] = "Bearer " + jwt_token

# Submit the request
response = requests.post(EAPI_ENDPOINT, 
                         headers = request_headers, 
                         data = json.dumps(payload))

For more information about allowable parameters for the payload, please see [Emissions API Developer Guide]().

In [5]:
if response.text != "":
    # Get the response as json
    response_json = response.json()
    
    # Get json and convert to dataframe
    json_str = json.dumps(response_json)
    dict = json.loads(json_str)
    dataframe = pd.json_normalize(dict) 
    
    # display
    print("\n\n")
    pd.set_option('display.max_colwidth', None)
    display( dataframe) 
else:
    print("Empty Response") 

,transactionId,totalCO2e,CO2,CH4,N2O,unit,description
0,fed4e869-b31a-4ebc-9ee0-b7dd1cc9a905,208.85354,206.92926,0.18006,1.74421,kgCO2e,The Business flights emissions factor used to calculate this result was obtained from the Managed - eGRID & US Climate Leaders factor set for the area Earth year 2024.


### Output Explanation

<b>transactionId</b> - An Emissions API transaction id.

<b>totalCO2e</b>     - The total emissions as CO<sub>2</sub> equivalent (CO<sub>2</sub>e)

<b>co2</b>           - The amount of CO<sub>2</sub> (Carbon Dioxide) in the CO<sub>2</sub>e value.

<b>ch4</b>           - The amount of CH<sub>4</sub> (Methane) in the CO<sub>2</sub>e value.

<b>n2O</b>           - The amount of N<sub>2</sub>O (Nitrous Oxide) in the CO<sub>2</sub>e value.

<b>hfc</b>           - The amount of HFCs (Hydrofluorocarbons) in the CO<sub>2</sub>e value.

<b>pfc</b>           - The amount of PFCs (Perfluorocarbons) in the CO<sub>2</sub>e value.

<b>sf6</b>           - The amount of SF<sub>6</sub> (Sulphur Hexafluoride) in the CO<sub>2</sub>e value.

<b>nf3</b>           - The amount of NF<sub>3</sub> (Nitrogen Trifluoride) in the CO<sub>2</sub>e value.

<b>bioCo2</b>        - The amount of bio CO<sub>2</sub> in the CO<sub>2</sub> value.

<b>indirectCo2e</b>  - The amount of CO<sub>2</sub>e that is indirect in the CO<sub>2</sub>e value.

<b>unit</b>          - The unit of measure of the values.

<b>description</b>   - A description of the source factor set of the factor used in the calculation.

## Related Links


[Emissions API Developer Guide](https://developer.ibm.com/apis/catalog/ghgemissions--ibm-envizi-emissions-api/Introduction)